In [35]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve

In [25]:
users = pd.read_table('lastfm-dataset-1K/userid-profile.tsv', header=0)
data = pd.read_table('lastfm-dataset-1K/userid-timestamp-artid-artname-traid-traname.tsv', header=-1,
                     #nrows=20000000,
                     error_bad_lines=False)

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [26]:
data = data.rename(columns={0:'userid', 1:'timestamp', 2:'artistid', 3:'artistname', 4:'trackid', 5:'trackname'})

data['timestamp'] = pd.to_datetime(data['timestamp'])

For the mini-project, we are using a smaller dataset. The following transformations will convert the dataset to use number of plays as our metric, grouped by user and artist.

In [28]:
data = data.groupby(['userid', 'artistname']).size().reset_index(name='plays')

In [62]:
users = list(np.sort(data.userid.unique())) # Get our unique users
artists = list(data.artistname.unique()) # Get our unique artists
quantity = list(data.plays) # All of our plays

rows = data.userid.astype('category', categories = users).cat.codes 
# Get the associated row indices
cols = data.artistname.astype('category', categories = artists).cat.codes 
# Get the associated column indices
plays_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(users), len(quantity)))
plays_sparse

<992x897419 sparse matrix of type '<class 'numpy.int64'>'
	with 897419 stored elements in Compressed Sparse Row format>

In [38]:
# Sparsity of the matrix
matrix_size = plays_sparse.shape[0]*plays_sparse.shape[1] # Number of possible interactions in the matrix
num_plays = len(plays_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_plays/matrix_size))
sparsity

99.8991935483871

## Dataset Reduction

The sparsity of 99.899% is extremely sparse, even for matrices that are intended to be sparse. We experimented with removing all rare artists, which had minimal effect on the sparsity. Instead, we will include only the top 100 artists, which has some improvement on the sparsity.

In [99]:
rare_artists = data.query("plays < 6"). \
    groupby('artistname').size().reset_index(name='users_listening_to_artist'). \
    query("users_listening_to_artist < 10")
    
top100_artists = data.groupby('artistname')['plays'].sum().reset_index(name='plays'). \
    nlargest(100,'plays')

,artistname,users_listening_to_artist
0,! Europe - France - Cold Wave,1
1,! Germany - Surfbeat From Beyond The Iron Curtain,1
2,! Www.Polskie-Mp3.Tk ! Adam Mi,1
3,! Www.Polskie-Mp3.Tk ! Breakout,1
4,! Www.Polskie-Mp3.Tk ! Budka Suflera,1


In [122]:
reduced_data = data[data.artistname.isin(top100_artists['artistname'])]

print(reduced_data.shape, data.shape)

(43179, 3) (897419, 3)


In [123]:
users = list(np.sort(reduced_data.userid.unique())) # Get our unique users
artists = list(reduced_data.artistname.unique()) # Get our unique artists
quantity = list(reduced_data.plays) # All of our plays

rows = reduced_data.userid.astype('category', categories = users).cat.codes 
# Get the associated row indices
cols = reduced_data.artistname.astype('category', categories = artists).cat.codes 
# Get the associated column indices
plays_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(users), len(quantity)))

plays_sparse

<979x43179 sparse matrix of type '<class 'numpy.int64'>'
	with 43179 stored elements in Compressed Sparse Row format>

In [124]:
# Sparsity of the matrix
matrix_size = plays_sparse.shape[0]*plays_sparse.shape[1] # Number of possible interactions in the matrix
num_plays = len(plays_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_plays/matrix_size))
sparsity

99.89785495403473

While 99.898% sparsity is not great, it is an improvement over the previous matrix, and so we will use this to set up our brute force model.